# Searching for faster matrix multiplication algorithms with quantum annealing

The notebook 2x2.ipynb implements an algorithm which translates higher order polynomials into quadratic format an unefficient way. This implementation aims to solve the problem.

In [7]:
import os
import sys
import dimod
import matplotlib.pyplot as plt
import numpy as np

this_folder = os.path.dirname(os.path.abspath(os.getcwd()))
sys.path.insert(0, this_folder)

from utils import *
from optimized_qubos import *

dim = 2
initial_tensor = get_standard_tensor(dim) % 2
origo = np.tensordot([0]*4, np.tensordot([0]*4, [0]*4, axes=0), axes=0)

In [8]:
strassen_tensors = [np.tensordot([0,0,0,1], np.tensordot([-1,0,1,0], [1,0,1,0], axes=0), axes=0),
          np.tensordot([1,1,0,0], np.tensordot([0,0,0,1], [-1,1,0,0], axes=0), axes=0),
           np.tensordot([-1,0,1,0], np.tensordot([1,1,0,0], [0,0,0,1], axes=0), axes=0),
           np.tensordot([1,0,0,1], np.tensordot([1,0,0,1], [1,0,0,1], axes=0), axes=0),
          np.tensordot([0,1,0,-1], np.tensordot([0,0,1,1], [1,0,0,0], axes=0), axes=0),
           np.tensordot([1,0,0,0], np.tensordot([0,1,0,-1], [0,1,0,1], axes=0), axes=0),
           np.tensordot([0,0,1,1], np.tensordot([1,0,0,0], [0,0,1,-1], axes=0), axes=0)]

The algorithm requires two good guesses: the tensor `np.tensordot([1,0,0,1], np.tensordot([1,0,0,1], [1,0,0,1], axes=0), axes=0)` and the initial tensor which is calculated in the beginning of the next cell.

In [9]:
tensor = (initial_tensor - strassen_tensors[0] - strassen_tensors[1] - strassen_tensors[2] - strassen_tensors[3]) % 2

# Moving towards origo
while(True):
    bqm = towards_user_defined_small(tensor, origo, dim)
    sample, energy, sampleset = solve_bqm_in_leap(bqm, "Greedy")
    #print(sampleset)
    x1, y1, z1 = process_result(sample, 2)
    print(x1, y1, z1)
    tensor = (tensor - np.tensordot(x1, np.tensordot(y1, z1, axes=0), axes=0)) % 2
    if np.count_nonzero(tensor.flatten()) == 0:
        print("End")
        break
        
tensor = (initial_tensor + strassen_tensors[0] + strassen_tensors[1] + strassen_tensors[2]) % 2
# Moving towards standard matrix multiplication i.e. the naive method
while(True):
    bqm = towards_user_defined_small(tensor, initial_tensor, dim)
    sample, energy, sampleset = solve_bqm_in_leap(bqm, "Greedy")
    #print(sampleset)
    x1, y1, z1 = process_result(sample, 2)
    print(x1, y1, z1)
    tensor = (tensor - np.tensordot(x1, np.tensordot(y1, z1, axes=0), axes=0)) % 2
    if np.array_equal(tensor, initial_tensor):
        print("End")
        break

Energy:  -4.0
Number of errors: 0
[1 0 0 0] [0 1 0 1] [0 1 0 1]
Energy:  -4.0
Number of errors: 0
[0 0 1 1] [1 0 0 0] [0 0 1 1]
Energy:  -4.0
Number of errors: 0
[0 1 0 1] [0 0 1 1] [1 0 0 0]
End
Energy:  -4.0
Number of errors: 0
[0 0 0 1] [1 0 1 0] [1 0 1 0]
Energy:  -4.0
Number of errors: 0
[1 1 0 0] [0 0 0 1] [1 1 0 0]
Energy:  -4.0
Number of errors: 0
[1 0 1 0] [1 1 0 0] [0 0 0 1]
End
